In [1]:
import glob
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import os
import time
from urllib.parse import urlparse
import random

all_txt_files = glob.glob(os.getcwd()+'\\prod_urls\\*.txt')

In [2]:
home_dir = os.getcwd()
results_dir = home_dir + '\\results'
images_dir = results_dir + '\\images'
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

In [3]:
all_txt_files

['E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Arts, Language and Linguistic Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Biographies, Memoirs and General Non-Ficton Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Fiction Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Healthy Living and Wellness Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\History and Archaeology Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Literature Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Philosophy and Religion Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Self-Help Books.txt']

In [4]:
genres_and_links = dict()

for filename in all_txt_files:
    with open(filename,'r') as file:
        relative_filename = filename.split('\\')[-1]
        genres_and_links[relative_filename.replace('.txt','')] = file.readlines()

In [5]:
for kk,vv in genres_and_links.items():
    print(len(vv),'-',kk)

1000 - Arts, Language and Linguistic Books
1000 - Biographies, Memoirs and General Non-Ficton Books
2000 - Fiction Books
1000 - Healthy Living and Wellness Books
2000 - History and Archaeology Books
2000 - Literature Books
1000 - Philosophy and Religion Books
2000 - Self-Help Books


In [6]:
dumb_urls = ['https://www.thepythoncode.com/','https://www.geeksforgeeks.org/','https://pynative.com/','http://facebook.com/','https://www.amazon.in/','https://www.tutorialspoint.com/','https://docs.python.org/','https://chromedriver.chromium.org/','https://github.com/','https://twitter.com/']

In [7]:
def replace_multispaces_with_single_space(text):
    text = re.sub('\\s+',' ',text)
    return text

In [8]:
ultimate_dataframes = dict()

In [9]:
driver = None

for genre, links in genres_and_links.items():
    
    results_dict = dict()

    list_of_book_titles = list()
    list_of_prices = list()
    list_of_publishers = list()
    list_of_pages = list()
    list_of_isbns = list()
    list_of_genres = list()
    list_of_ratings = list()
    list_of_authors = list()
    list_of_book_summaries = list()
    list_of_image_names = list()

    main_counter = 0
    for link in links:
        
        requests.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
        #time.sleep(1)
        resp = requests.get(link)
        
        soup = BeautifulSoup(resp.content,'html.parser')
        
        book_title_ele = soup.find('span',{'class':'_35KyD6'})
        book_title = book_title_ele.get_text()
        book_title = replace_multispaces_with_single_space(book_title)
        
        
        price = soup.find('div',{'class':'_1vC4OE _3qQ9m1'}).get_text()
        price = re.sub('₹','',price)
        
        spec_tbody_elements = soup.find_all('tr',{'class':'_3_6Uyw row'})
        publisher = 'NA'
        for tbody in spec_tbody_elements:
            tr_elements = tbody.find_all('td')
            if len(tr_elements)==2:
                if tr_elements[0] and tr_elements[0].text=='Imprint':
                    publisher = tr_elements[1].get_text()
    
        no_of_pages = 'NA'
        isbn = 'NA'
        highlights_li_elements = soup.find_all('li',{'class':'_2-riNZ'})
        for li in highlights_li_elements:
            if li and li.get_text() and li.get_text().startswith('Pages'):
                no_of_pages = li.get_text().split(': ')[1]
            if li and li.get_text() and li.get_text().startswith('ISBN'):
                isbn = li.get_text().split(': ')[1]
        
        rating = 'NA'
        rating_parent_element = soup.find('div',{'class':'_3ors59'})
        if rating_parent_element:
            rating_element = rating_parent_element.find('div',{'class':'hGSR34'})
            if rating_element:
                rating = rating_element.get_text()
        
        author = 'NA'
        book_summary = 'NA'
        book_desc_list = list()
        author_description_elements = soup.find_all('div',{'class':'_1y9a40'})
        for element in author_description_elements:
            if element.find('div',{'class':'_1oCqc9'}) and element.find('div',{'class':'_1oCqc9'}).get_text()=='Author':
                author = element.find('div',{'class':'_3cpW1u'}).get_text().strip()
            if element.find('div',{'class':'_1oCqc9'}) and element.find('div',{'class':'_1oCqc9'}).get_text()=='Description':
                if element.find('div',{'class':'_3cpW1u'}):
                    book_summary = element.find('div',{'class':'_3cpW1u'}).get_text()
                    book_summary = replace_multispaces_with_single_space(book_summary)
                #desc_p_elements = element.find_all('p');
                #book_desc_list = [re.sub('\\s+',' ',p.get_text()) for p in desc_p_elements if p]
                #book_summary = ''.join(book_desc_list)
                
        #print(help(soup.find('div',{'class':'_2rDnao'})))
        #print(help(soup.find('div',{'class':'_2rDnao'}).get_attribute_list))
        
        image_urls = set()
        image_elements = soup.find_all('div',{'class':'_2_AcLJ'})
        image_url = None
        for ele in image_elements:
            if ele.has_attr('style'):
                if '(' in ele.get('style') and ')' in ele.get('style'):
                    image_url = ele.get('style').split('(')[1].split(')')[0]
                if image_url is not None:
                    image_url = re.sub('\\w+?q=\\d+','',image_url)
                    image_url = re.sub('https://rukminim1.flixcart.com/image/\\d+/\\d+/','https://rukminim1.flixcart.com/image/512/512/',image_url)
                    url_parse_results = urlparse(image_url)
                    if all([url_parse_results.scheme, url_parse_results.netloc, url_parse_results.path]):
                        image_urls.add(image_url)
                
        if len(image_urls)==0:
            if driver is None:
                driver = webdriver.Chrome(executable_path='F:\\Development\\classpath\\selenium_drivers\\chromedriver.exe')
            driver.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
            #time.sleep(1)
            driver.get(link)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source,'html.parser')
            image_divs = soup.find_all('div',{'class':'_1ov7-N'})
            for image_div in image_divs:
                img_elements = image_div.find_all('img')
                for img in img_elements:
                    if img.has_attr('src'):
                        image_url = img.get('src')
            if image_url is not None:
                image_url = re.sub('\\w+?q=\\d+','',image_url)
                image_url = re.sub('https://rukminim1.flixcart.com/image/\\d+/\\d+/','https://rukminim1.flixcart.com/image/512/512/',image_url)
                url_parse_results = urlparse(image_url)
                if all([url_parse_results.scheme, url_parse_results.netloc, url_parse_results.path]):
                    image_urls.add(image_url)
                
        i_counter = 1
        i_names = list()
        for i_url in image_urls:
            genre_images_dir = results_dir + '\\images\\' + genre
            if not os.path.exists(genre_images_dir):
                os.makedirs(genre_images_dir)
            i_name = genre_images_dir + '\\' +str(main_counter+1)+'.'+str(i_counter)+'.jpeg'
            
            requests.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
            #time.sleep(1)
            i_response = requests.get(i_url)
                
            with open(i_name, 'wb') as f:
                f.write(i_response.content)
            i_name = '\\'.join([i_name.split('\\')[-4],i_name.split('\\')[-3],i_name.split('\\')[-2],i_name.split('\\')[-1]])
            i_names.append(i_name)
            #print(i_name)
            i_counter +=1
        

        #print(book_title)#'title_bindtype_language-'
        #print(price)
        #print(publisher)
        #print(no_of_pages)
        #print(isbn)
        #print(genre)
        #print(rating)
        #print(author)
        #print(book_summary)
        #print(image_urls)
        
        list_of_book_titles.append(book_title)
        list_of_prices.append(price)
        list_of_publishers.append(publisher)
        list_of_pages.append(no_of_pages)
        list_of_isbns.append(isbn)
        list_of_genres.append(genre)
        list_of_ratings.append(rating)
        list_of_authors.append(author)
        list_of_book_summaries.append(book_summary)
        list_of_image_names.append(', '.join(i_names))
        
        #print('-----------------')
        
        main_counter += 1
        if main_counter==10:
            break;
    
    results_dict['book_title'] = list_of_book_titles
    results_dict['price'] = list_of_prices
    results_dict['publisher'] = list_of_publishers
    results_dict['no_of_pages'] = list_of_pages
    results_dict['isbn'] = list_of_isbns
    results_dict['genre'] = list_of_genres
    results_dict['rating'] = list_of_ratings
    results_dict['author'] = list_of_authors
    results_dict['book_summary'] = list_of_book_summaries
    results_dict['images'] = list_of_image_names
    
    df = pd.DataFrame(results_dict)
    export_file_name = results_dir + '\\' + genre + '.csv'
    df.to_csv(export_file_name,index=False)
    ultimate_dataframes[genre] = df

In [10]:
if driver is not None:
    driver.quit()

In [11]:
list(ultimate_dataframes.values())[0].head()

book_title price  \
0  Objective General English (English, Paperback,...   203   
1  Word Power Made Easy - The Complete Handbook f...   140   
2  Essential English Grammar (English, Paperback,...   142   
3  Englishwale.com English Speaking Course Book (...   225   
4  The Official Cambridge Guide to Ielts Student'...   428   

                    publisher no_of_pages                           isbn  \
0          Arihant Publishers         755   9789351768449, 9789351768449   
1                     Penguin         560   9780143424680, 9780143424680   
2            Foundation Books         300   9788175960299, 9788175960299   
3                          NA         400  9788193074305, 9788193074305,   
4  Cambridge University Press         400   9781107442771, 9781107442771   

                                 genre rating          author  \
0  Arts, Language and Linguistic Books    4.4    Bakshi S. P.   
1  Arts, Language and Linguistic Books    4.4    Lewis Norman   
2  Arts, Language and Linguistic Books    4.3          Murphy   
3  Arts, Language and Linguistic Books    4.3      Pooja Rana   
4  Arts, Language and Linguistic Books    4.4  Cullen Pauline   

                                        book_summary  \
0  About the Book Objective General English is an...   
1                                                 NA   
2   Essential English Grammar: A Self-Study Refer...   
3  Designed & Developed by EnglishWale.com in ass...   
4  Definitive preparation for IELTS - from the ma...   

                                              images  
0  results\images\Arts, Language and Linguistic B...  
1  results\images\Arts, Language and Linguistic B...  
2  results\images\Arts, Language and Linguistic B...  
3  results\images\Arts, Language and Linguistic B...  
4  results\images\Arts, Language and Linguistic B...

In [12]:
df1 = list(ultimate_dataframes.values())[0]
df1['images'].unique()

array(['results\\images\\Arts, Language and Linguistic Books\\1.1.jpeg',
       'results\\images\\Arts, Language and Linguistic Books\\2.1.jpeg',
       'results\\images\\Arts, Language and Linguistic Books\\3.1.jpeg',
       'results\\images\\Arts, Language and Linguistic Books\\4.1.jpeg',
       'results\\images\\Arts, Language and Linguistic Books\\5.1.jpeg',
       'results\\images\\Arts, Language and Linguistic Books\\6.1.jpeg',
       'results\\images\\Arts, Language and Linguistic Books\\7.1.jpeg',
       'results\\images\\Arts, Language and Linguistic Books\\8.1.jpeg, results\\images\\Arts, Language and Linguistic Books\\8.2.jpeg',
       'results\\images\\Arts, Language and Linguistic Books\\9.1.jpeg',
       'results\\images\\Arts, Language and Linguistic Books\\10.1.jpeg'],
      dtype=object)